In [2]:
from manim import *
import numpy as np

In [11]:
%%manim -qm -v WARNING QuadraticFunction

class Count(Animation):
    def __init__(self, number: DecimalNumber, start: float, end: float, **kwargs) -> None:
        # Pass number as Mobject to Super
        self.rate_func = kwargs.pop('rate_func', smooth)
        super().__init__(number, **kwargs)
        # Set parameters
        self.start = start
        self.end = end
    
    def interpolate_mobject(self, alpha) -> None:
        # Set value to be incremental using alpha
        rate_func_alpha = self.rate_func(alpha)
        value = self.start + (rate_func_alpha * (self.end - self.start))
        self.mobject.set_value(value)

class TransformDot(Animation):
    def __init__(self, dot: Dot, start_graph: [float], end_graph: [float], **kwargs) -> None:
        self.rate_func = kwargs.pop('rate_func', smooth)
        super().__init__(dot, **kwargs)
        self.start_graph = start_graph
        self.end_graph = end_graph

    def interpolate_mobject(self, alpha) -> None:

        rate_func_alpha = self.rate_func(alpha)
        alpha_a = self.start_graph[0] + ((self.end_graph[0] - self.start_graph[0])*rate_func_alpha)
        alpha_b = self.start_graph[1] + ((self.end_graph[1] - self.start_graph[1])*rate_func_alpha)
        alpha_c = self.start_graph[2] + ((self.end_graph[2] - self.start_graph[2])*rate_func_alpha)

        alpha_x = -alpha_b/(2*alpha_a)
        alpha_y = (alpha_a*(alpha_x**2))+(alpha_b*alpha_x) + alpha_c
        
        self.mobject.move_to(np.array([alpha_x, alpha_y, 0]))
        
        
class QuadraticFunction(Scene):
    def construct(self):
        # Set background
        number_plane = NumberPlane()
        border_m_disp = Rectangle(width=3.0, height=2.0).shift(RIGHT*5, UP*2.5).set_fill(BLACK, opacity=1)
        # 
        def get_quad_func(a, b, c):
            return FunctionGraph(
                lambda x: (a*(x**2))+(b*x) + c,
                z_index=-1
            )
    
        def get_value_point(a, b, c):
            x = - b/(2*a)
            y = (a*(x**2))+(b*x) + c
            return number_plane.c2p(x, y)
            
        def get_line(one:[float], two:[float]):
            points_along_path_b = np.linspace(one, two, 100)
            points_along_path = [get_value_point(*abc) for abc in points_along_path_b]
            line = VMobject()
            line.set_points_smoothly(points_along_path)
            return line
            
        qforg = get_quad_func(1, 3, 0)
        qfone = get_quad_func(1, 3, 0)
        qftwo = get_quad_func(1, -3, 0)
        qfthr = get_quad_func(5, -3, 0)

        line_qfone_to_qftwo = get_line(3, -3)
        line_qfone_to_qfone = get_line(-3, 3)

        
        dot = Dot(get_value_point(1, 3, 0))

        a_number = DecimalNumber(1)
        b_number = DecimalNumber(3)
        c_number = DecimalNumber(0)

        label_a = MathTex(r"a = ")
        label_b = MathTex(r"b = ")
        label_c = MathTex(r"c = ")
        
        counter_a = VGroup(label_a, a_number).arrange(RIGHT, aligned_edge=DOWN)
        counter_b = VGroup(label_b, b_number).arrange(RIGHT, aligned_edge=DOWN)
        counter_c = VGroup(label_c, c_number).arrange(RIGHT, aligned_edge=DOWN)
        
        counters = VGroup(counter_a, counter_b, counter_c).arrange(DOWN, aligned_edge=LEFT).next_to(border_m_disp, ORIGIN)
        
        self.add(number_plane, border_m_disp, counter_b, counter_a, counter_c)
        self.add(qforg, dot)
        self.play(
            Count(b_number, 3,  -3), 
            Transform(qforg, qftwo), 
            Create(line_qfone_to_qftwo), 
            TransformDot(dot, [1, 3, 0], [1, -3, 0], rate_func=smooth, run_time=2), 
            rate_func=smooth, 
            run_time=2
        )
        self.wait()
        self.play(
            Count(b_number, -3,  3), 
            Transform(qforg, qfone), 
            Uncreate(line_qfone_to_qftwo), 
            TransformDot(dot, [1, -3, 0], [1, 3, 0], rate_func=smooth, run_time=2), 
            rate_func=smooth, 
            run_time=2
        )
        self.wait()
        self.play(
            Count(a_number, 1,  5), 
            Transform(qforg, qfone), 
            Uncreate(line_qfone_to_qftwo), 
            TransformDot(dot, [1, -3, 0], [5, -3, 0], rate_func=smooth, run_time=2), 
            rate_func=smooth, 
            run_time=2
        )
        

Manim Community v0.18.1

TypeError: __main__.QuadraticFunction.construct.<locals>.get_value_point() argument after * must be an iterable, not numpy.float64